In [1]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dropout
from keras.preprocessing import sequence
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import gc
import scipy


Using TensorFlow backend.


In [2]:
os.getcwd()
os.chdir("C:\\Users\\srita\\Python\\Kaggle\\Mercari Price Challenge")

In [3]:
#Read the data
df = pd.read_csv("train.tsv",delimiter='\t',encoding='utf-8')

In [4]:
# Sample the data to speed up computation
df = df.sample(frac=0.1, random_state=10)

df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
774893,774893,Mario and luigi,1,Kids/Toys/Dress Up & Pretend Play,NaN,18.0,1,Girls costumes. I can separate or i will sell ...
431736,431736,Air Jordan Baron 9,3,Men/Shoes/Athletic,Jordan,91.0,0,Sz 10.5 Good preowned condition Includes origi...
1482299,1482299,Dooney & Bourke Wallet,3,Women/Women's Accessories/Wallets,Dooney & Bourke,20.0,1,D&B wallet with credit card slots and back coi...
764342,764342,Lululemon Floral Waistband Speed Tights,3,"Women/Athletic Apparel/Pants, Tights, Leggings",Lululemon,66.0,0,This is a pair of size 6 speed tights in great...
781985,781985,American Eagle Top,2,Women/Athletic Apparel/Shirts & Tops,American Eagle,19.0,0,AEO off the shoulder top size XL


In [5]:
# Drop missing values
df.dropna(inplace=True)

df.dropna(inplace=True)
df.head(10)
df.set_index('train_id')

#df['Good Condition'] = np.where(df['item_condition_id'] > 2, 1,0)


,name,item_condition_id,category_name,brand_name,price,shipping,item_description
train_id,,,,,,,
431736,Air Jordan Baron 9,3,Men/Shoes/Athletic,Jordan,91.0,0,Sz 10.5 Good preowned condition Includes origi...
1482299,Dooney & Bourke Wallet,3,Women/Women's Accessories/Wallets,Dooney & Bourke,20.0,1,D&B wallet with credit card slots and back coi...
764342,Lululemon Floral Waistband Speed Tights,3,"Women/Athletic Apparel/Pants, Tights, Leggings",Lululemon,66.0,0,This is a pair of size 6 speed tights in great...
781985,American Eagle Top,2,Women/Athletic Apparel/Shirts & Tops,American Eagle,19.0,0,AEO off the shoulder top size XL
590192,Nike crop leggings size medium,3,"Women/Athletic Apparel/Pants, Tights, Leggings",Nike,12.0,1,Black and white pattern Gently used condition ...
1244225,Victoria's Secret Makeup Bag - NEW!,1,Women/Women's Handbags/Cosmetic Bags,Victoria's Secret,7.0,1,Includes: Victoria's Secret Makeup Bag - New w...
1308385,Starbucks holiday cozy cup sleeve,1,Home/Kitchen & Dining/Coffee & Tea Accessories,Starbucks,8.0,1,Perfect holiday gift that goes great with a mu...
124724,Lego 76040 Brainiac Attack,1,Kids/Toys/Building Toys,LEGO,37.0,1,No description yet
605273,Clinique HAPPY perfume gift set,1,Beauty/Fragrance/Women,Clinique,80.0,1,Free priority shipping included! Super deluxe ...


In [6]:
Y = df['price']


In [7]:
mx = np.max(Y)
print(mx)

2009.0


In [8]:
val = np.array([0.2,1.4,2.4,4.5,8,9,2.2])
bins_set = pd.cut(val, 3,labels = ['small', 'medium','high'])

In [9]:
bins_df = pd.DataFrame(val,bins_set)

In [10]:
bins_df

,0
small,0.2
small,1.4
small,2.4
medium,4.5
high,8.0
high,9.0
small,2.2


In [11]:
test = np.array(range(7))
bins_df['test'] = test
bins_df

,0,test
small,0.2,0
small,1.4,1
small,2.4,2
medium,4.5,3
high,8.0,4
high,9.0,5
small,2.2,6


In [12]:
price_bins = pd.cut(Y, 10, labels = ['LVLow','UVLow','LLow','ULow','LMedium','UMedium','LHigh','UHigh','LVHigh','UVHigh'])

In [13]:
df['price_bins'] = price_bins

In [14]:
print("Number of items less than 200",len(df.loc[(df['price_bins'] == 'LVLow')]))
print("Number of items between 200 and 400",len(df.loc[(df['price_bins'] == 'UVLow')]))
print("Number of items between 400 and 600",len(df.loc[(df['price_bins'] == 'LLow')]))
print("Number of items between 600 and 800",len(df.loc[(df['price_bins'] == 'ULow')]))
print("Number of items between 800 and 1000",len(df.loc[(df['price_bins'] == 'LMedium')]))
print("Number of items between 1000 and 1200",len(df.loc[(df['price_bins'] == 'UMedium')]))
print("Number of items between 1200 and 1400",len(df.loc[(df['price_bins'] == 'LHigh')]))
print("Number of items between 1400 and 1600",len(df.loc[(df['price_bins'] == 'UHigh')]))
print("Number of items between 1600 and 1800",len(df.loc[(df['price_bins'] == 'LVHigh')]))
print("Number of items between 1800 and above",len(df.loc[(df['price_bins'] == 'UVHigh')]))

Number of items less than 200 83767
Number of items between 200 and 400 675
Number of items between 400 and 600 109
Number of items between 600 and 800 28
Number of items between 800 and 1000 16
Number of items between 1000 and 1200 6
Number of items between 1200 and 1400 3
Number of items between 1400 and 1600 2
Number of items between 1600 and 1800 0
Number of items between 1800 and above 4


In [15]:
df.columns

Index(['train_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'price', 'shipping', 'item_description', 'price_bins'],
      dtype='object')

In [16]:
X = df.drop('train_id', axis=1)
X = df.drop('name',axis=1)

In [17]:
X.drop('train_id',axis=1,inplace=True)

In [18]:
#X is a dataframe contains required predictors, (item_condition_id, category_name, brand_name, price, shipping, item_description)
#Create a new data frame X_noprice without price, including price_bins categorical
X_noprice = X.drop('price',axis=1)


In [19]:
X.head()
X_noprice.head()

,item_condition_id,category_name,brand_name,shipping,item_description,price_bins
431736,3,Men/Shoes/Athletic,Jordan,0,Sz 10.5 Good preowned condition Includes origi...,LVLow
1482299,3,Women/Women's Accessories/Wallets,Dooney & Bourke,1,D&B wallet with credit card slots and back coi...,LVLow
764342,3,"Women/Athletic Apparel/Pants, Tights, Leggings",Lululemon,0,This is a pair of size 6 speed tights in great...,LVLow
781985,2,Women/Athletic Apparel/Shirts & Tops,American Eagle,0,AEO off the shoulder top size XL,LVLow
590192,3,"Women/Athletic Apparel/Pants, Tights, Leggings",Nike,1,Black and white pattern Gently used condition ...,LVLow


In [20]:
X_noprice.dtypes
X_noprice['item_condition_id'] = X_noprice['item_condition_id'].astype('category')
X_noprice['shipping'] = X_noprice['shipping'].astype('category')

In [21]:
#Split the category_name to sub categories
X_noprice['category_name'], X_noprice['subcat_1'], X_noprice['subcat_2'] = zip(*X_noprice['category_name'].apply(lambda x: x.split('/')))

In [22]:
X_noprice.head()

,item_condition_id,category_name,brand_name,shipping,item_description,price_bins,subcat_1,subcat_2
431736,3,Men,Jordan,0,Sz 10.5 Good preowned condition Includes origi...,LVLow,Shoes,Athletic
1482299,3,Women,Dooney & Bourke,1,D&B wallet with credit card slots and back coi...,LVLow,Women's Accessories,Wallets
764342,3,Women,Lululemon,0,This is a pair of size 6 speed tights in great...,LVLow,Athletic Apparel,"Pants, Tights, Leggings"
781985,2,Women,American Eagle,0,AEO off the shoulder top size XL,LVLow,Athletic Apparel,Shirts & Tops
590192,3,Women,Nike,1,Black and white pattern Gently used condition ...,LVLow,Athletic Apparel,"Pants, Tights, Leggings"


In [23]:
X_noprice.dtypes

item_condition_id    category
category_name          object
brand_name             object
shipping             category
item_description       object
price_bins           category
subcat_1               object
subcat_2               object
dtype: object

In [24]:
target = X_noprice['price_bins']


In [25]:
X_noprice = X_noprice.drop('price_bins',axis=1)

In [26]:
X_noprice.shape

(84610, 7)

In [27]:
tfidf_transformer = TfidfVectorizer(max_features=50000,ngram_range=(1,1),stop_words='english')

In [28]:
item_desc_vect = tfidf_transformer.fit_transform(X_noprice['item_description'])
brand_vect = tfidf_transformer.fit_transform(X_noprice['brand_name'])
cat_vect = tfidf_transformer.fit_transform(X_noprice['category_name'])
subcat_1_vect = tfidf_transformer.fit_transform(X_noprice['subcat_1'])
subcat_2_vect = tfidf_transformer.fit_transform(X_noprice['subcat_2'])

In [29]:
type(subcat_1_vect)

scipy.sparse.csr.csr_matrix

In [30]:
X_dummies = scipy.sparse.csr_matrix(pd.get_dummies(X_noprice[["item_condition_id", "shipping"]], sparse=True).values)

In [31]:
print("Item description shape:",item_desc_vect.shape)
print("brand name shape:",brand_vect.shape)
print("category name shape:",cat_vect.shape)
print("subcategory 1 shape:",subcat_1_vect.shape)
print("subcategory 2 shape:",subcat_2_vect.shape)

Item description shape: (84610, 33656)
brand name shape: (84610, 2634)
category name shape: (84610, 11)
subcategory 1 shape: (84610, 117)
subcategory 2 shape: (84610, 620)


In [32]:
X = scipy.sparse.hstack((X_dummies,item_desc_vect,brand_vect,cat_vect,subcat_1_vect,subcat_2_vect)).tocsr()

In [33]:
# X is Combined matrix of all the vectors
X.shape

(84610, 37045)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,target,random_state=0)

In [35]:
print("Train shape:",X_train.shape)
print("Test shape:",X_test.shape)

Train shape: (63457, 37045)
Test shape: (21153, 37045)


In [36]:
target.shape
y_train.shape
y_test.shape

(21153,)

In [37]:
y_train.head()

1312383    LVLow
375433     LVLow
907837     LVLow
1025567     LLow
937808     LVLow
Name: price_bins, dtype: category
Categories (10, object): [LVLow < UVLow < LLow < ULow ... LHigh < UHigh < LVHigh < UVHigh]

## Modelling

In [38]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)
pred=clf.predict(X_train)
print(y_train.value_counts())
from sklearn.metrics import confusion_matrix
prediction=confusion_matrix(y_train,pred)


LVLow      62822
UVLow        506
LLow          81
ULow          22
LMedium       15
UVHigh         4
UMedium        3
UHigh          2
LHigh          2
LVHigh         0
Name: price_bins, dtype: int64


In [39]:
print(prediction)

[[    0     0     0     2     0     0     0     0     0]
 [    0     0     0    78     0     0     0     0     3]
 [    0     0     0    15     0     0     0     0     0]
 [    0     0     0 62807     0     0     0     0    15]
 [    0     0     0     2     0     0     0     0     0]
 [    0     0     0    21     0     0     0     0     1]
 [    0     0     0     3     0     0     0     0     0]
 [    0     0     0     4     0     0     0     0     0]
 [    0     0     0   495     0     0     0     0    11]]


In [40]:
#Metrics
from sklearn.metrics import recall_score, precision_score, accuracy_score
acc=accuracy_score(y_train,pred)
rec=recall_score(y_train,pred,labels=['LVLow','UVLow','LLow','ULow','LMedium','UVHigh','UMedium','UHigh','LHigh','LVHigh'],average='micro')
prec=precision_score(y_train,pred,labels=['LVLow','UVLow','LLow','ULow','LMedium','UVHigh','UMedium','UHigh','LHigh','LVHigh'],average='micro')


In [41]:
print("Accuracy using Micro",acc*100)
print("Recall: using Micro",rec*100)
print("Precision:using Micro",prec*100)

Accuracy using Micro 98.9930188947
Recall: using Micro 98.9930188947
Precision:using Micro 98.9930188947


In [42]:
#Predictions on test data, X_test
test_preds = clf.predict(X_test)

In [43]:
print(test_preds)

['LVLow' 'LVLow' 'LVLow' ..., 'LVLow' 'LVLow' 'LVLow']


In [44]:
test_predictions = confusion_matrix(y_test,test_preds)
print(test_predictions)
print(y_test.value_counts())

[[    0     0     0     1     0     0     0]
 [    0     0     0    28     0     0     0]
 [    0     0     0     1     0     0     0]
 [    0     0     0 20938     0     0     7]
 [    0     0     0     6     0     0     0]
 [    0     0     0     3     0     0     0]
 [    0     0     0   168     0     0     1]]
LVLow      20945
UVLow        169
LLow          28
ULow           6
UMedium        3
LHigh          1
LMedium        1
UVHigh         0
LVHigh         0
UHigh          0
Name: price_bins, dtype: int64


In [45]:
acc=accuracy_score(y_test,test_preds)
rec=recall_score(y_test,test_preds,labels=['LVLow','UVLow','LLow','ULow','LMedium','UVHigh','UMedium','UHigh','LHigh','LVHigh'],\
                 average='micro')
prec=precision_score(y_test,test_preds,labels=['LVLow','UVLow','LLow','ULow','LMedium','UVHigh','UMedium','UHigh','LHigh','LVHigh'],\
                     average='micro')
print("Test Accuracy using Micro",acc*100)
print("Test Recall: using Micro",rec*100)
print("Test Precision:using Micro",prec*100)

Test Accuracy using Micro 98.9883231693
Test Recall: using Micro 98.9883231693
Test Precision:using Micro 98.9883231693


In [46]:
Results = pd.DataFrame()
Results['Train_Actuals'] = y_train
Results['Train_predictions'] = pred


In [47]:
Test_Results = pd.DataFrame()
Test_Results['Test_Actuals'] = y_test
Test_Results['Test_predictions'] = test_preds

In [48]:
Results.head()

,Train_Actuals,Train_predictions
1312383,LVLow,LVLow
375433,LVLow,LVLow
907837,LVLow,LVLow
1025567,LLow,LVLow
937808,LVLow,LVLow


In [49]:
Test_Results.head()

,Test_Actuals,Test_predictions
1074401,LVLow,LVLow
370950,LVLow,LVLow
469604,LVLow,LVLow
1014695,LVLow,LVLow
506488,LVLow,LVLow


## Regression Modelling - predicting target by considering price

In [50]:
X.shape

(84610, 37045)

In [51]:
target_p = df['price']

In [52]:
X_train,X_test,y_train,y_test = train_test_split(X,target_p,random_state=0)

In [53]:
X_train.shape

(63457, 37045)

In [54]:
X_test.shape

(21153, 37045)